# Data Preprocessing

In [66]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def load_and_shuffle_mnist():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5, ))])
    mnist_train = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    mnist_test = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

    train_data = list(mnist_train)
    test_data = list(mnist_test)

    random.shuffle(train_data)
    random.shuffle(test_data)

    return train_data, test_data

In [67]:
def create_sequences(mnist_data, window_size):
    sequences = []
    labels = []

    for i in range(len(mnist_data) - window_size + 1):
        window = mnist_data[i:i + window_size]
        sequence_images = torch.stack([img for img, _ in window])

        # Default label is 'null' (using a specific number to represent 'null', e.g., 10)
        label = 10  # Assuming 10 represents 'null'

        # Check for the condition in the last window_size elements
        last_digit = window[-1][1]
        for _, prev_label in window[:-1]:
            if prev_label == last_digit:
                label = last_digit  # Label is the digit itself if condition is met
                break

        sequences.append(sequence_images)
        labels.append(label)

    return sequences, labels

In [68]:
class MNISTSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels  # Labels are already numerical, no need for a map

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        # Ensure label is a tensor of dtype torch.long
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)
        return sequence, label_tensor

# LSMT Definition

In [69]:

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        x = x.view(-1, window_size, 28*28)  # Reshape x
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [70]:
def balanced_sampling(mnist_data, size, num_classes=11):
    print('balanced_sampling')
    # Group data by labels
    grouped_data = {label: [] for label in range(num_classes)}
    for img, label in mnist_data:
        grouped_data[label].append((img, label))

    # Calculate the number of samples per class
    samples_per_class = size // num_classes

    # Initialize list for sampled data
    sampled_data = []

    for label, data in grouped_data.items():
        data_len = len(data)
        if data_len == 0:
            print(f"Warning: No data for class {label}. Skipping this class.")
            continue
        if data_len >= samples_per_class:
            sampled_data.extend(random.sample(data, samples_per_class))
        else:
            # Efficiently replicate data to meet the required number of samples
            repeats = samples_per_class // data_len
            remainder = samples_per_class % data_len
            sampled_data.extend(data * repeats + random.sample(data, remainder))

    # Shuffle the final dataset using Python's random.shuffle for compatibility
    random.shuffle(sampled_data)
    print('Done sampling.')
    return sampled_data


# Train and Testing

In [64]:
import time
import numpy as np
from sklearn.metrics import f1_score


def train_model(model, train_loader, criterion, optimizer, num_epochs,test_loader):
    model = model.to(device)  # Move model to GPU

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        logger = {'time': {}, 'loss': {}, 'F1': {}, 'Accuracy': {}}
        print(f"Starting Epoch {epoch + 1}/{num_epochs}")  # Print at the start of each epoch


        for i, (sequences, labels) in enumerate(train_loader):
            sequences, labels = sequences.to(device), labels.to(device)  # Move data to GPU

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            if i % 100 == 0:
                logger['time'][i] = time.time() - epoch_start_time
                logger['loss'][i] = loss.item()
                print(f"Epoch {epoch + 1}, Step {i}, Loss: {loss.item()}, Time Elapsed: {time.time() - epoch_start_time} seconds")


        epoch_end_time = time.time()
        print(f"Epoch {epoch + 1}\n")
        print(f"Writing snapshot to model_iter_{(epoch + 1) * len(train_loader):06d}.mdl")
        print(f"Epoch time: {epoch_end_time - epoch_start_time}")

        # Evaluate at the end of the epoch
        f1_score, accuracy = evaluate_model(model, test_loader)
        logger['F1'][len(test_loader) * (epoch + 1)] = f1_score
        logger['Accuracy'][len(test_loader) * (epoch + 1)] = accuracy

        # Print detailed logger information
        print("Logger", logger)

# Update the evaluate_model function to return F1 score and accuracy
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in test_loader:
            outputs = model(sequences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels, all_predictions))

    # Compute F1 score and accuracy
    f1_score_value = f1_score(all_labels, all_predictions, average='macro')  # Change average as needed
    accuracy = correct / total  # Corrected accuracy calculation
    return f1_score_value, accuracy

In [61]:
# Load and shuffle MNIST dataset
train_data, test_data = load_and_shuffle_mnist()

# Set LSTM parameters
input_size = 784  # 28x28
hidden_size = 128
num_classes = 11  # Digits 0-9 and 'null'
num_epochs = 10
batch_size = 100
learning_rate = 0.001
window_size = 10  # Example window size

# Create model
model = SimpleLSTM(input_size, hidden_size, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Iterate over different dataset sizes
for size in [1000, 5000, 10000, 20000, 30000, 40000, 50000]:
    print(f"Training with dataset size: {size}")

    # Balanced sampling
    sampled_train_data = balanced_sampling(train_data, size)
    sampled_test_data = balanced_sampling(test_data, size // 2)
    print('Done sample data.')

    # Create sequences
    train_sequences, train_labels = create_sequences(sampled_train_data, window_size)
    test_sequences, test_labels = create_sequences(sampled_test_data, window_size)

    # Create data loaders
    train_dataset = MNISTSequenceDataset(train_sequences, train_labels)
    test_dataset = MNISTSequenceDataset(test_sequences, test_labels)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Train and evaluate the model
    train_model(model, train_loader, criterion, optimizer, num_epochs, test_loader)


Training with dataset size: 1000
balanced_sampling
Done sampling.
balanced_sampling
Done sampling.
Done sample data.
Starting Epoch 1/10
Epoch 1, Step 0, Loss: 2.2875936031341553, Time Elapsed: 0.1277780532836914 seconds
Epoch 1

Writing snapshot to model_iter_000009.mdl
Epoch time: 0.9980049133300781
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        22
           6       0.00      0.00      0.00        26
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.41      1.00      0.58       179

    accuracy                           0.41    

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Epoch 2

Writing snapshot to model_iter_000018.mdl
Epoch time: 0.6590173244476318
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        22
           6       0.00      0.00      0.00        26
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.41      1.00      0.58       179

    accuracy                           0.41       441
   macro avg       0.04      0.09      0.05       441
weighted avg       0.16      0.41      0.23       441

Logger {'time': {0: 0.06718325614929199}, 'loss': {0: 1.961031436920166}, 'F1': {10: 0.05249266862170088}

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Epoch 3

Writing snapshot to model_iter_000027.mdl
Epoch time: 0.4717540740966797
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        22
           6       0.50      0.04      0.07        26
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.41      0.99      0.58       179

    accuracy                           0.41       441
   macro avg       0.08      0.09      0.06       441
weighted avg       0.19      0.41      0.24       441

Logger {'time': {0: 0.05473208427429199}, 'loss': {0: 1.7852561473846436}, 'F1': {15: 0.05886185012398604

...

Epoch 4

Writing snapshot to model_iter_000036.mdl
Epoch time: 0.4778261184692383
              precision    recall  f1-score   support

           0       0.43      0.11      0.17        28
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        22
           6       0.60      0.12      0.19        26
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.40      0.97      0.57       179

    accuracy                           0.41       441
   macro avg       0.13      0.11      0.08       441
weighted avg       0.23      0.41      0.25       441

Logger {'time': {0: 0.04736614227294922}, 'loss': {0: 1.6081671714782715}, 'F1': {20: 0.0849141732630697}

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Epoch 5

Writing snapshot to model_iter_000045.mdl
Epoch time: 0.471494197845459
              precision    recall  f1-score   support

           0       0.67      0.29      0.40        28
           1       0.75      0.21      0.33        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        22
           6       0.67      0.31      0.42        26
           7       0.40      0.07      0.12        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.41      0.93      0.57       179

    accuracy                           0.43       441
   macro avg       0.26      0.16      0.17       441
weighted avg       0.32      0.43      0.31       441

Logger {'time': {0: 0.05804610252380371}, 'loss': {0: 1.4979350566864014}, 'F1': {25: 0.16807765749943238}

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Epoch 6

Writing snapshot to model_iter_000054.mdl
Epoch time: 0.47171783447265625
              precision    recall  f1-score   support

           0       0.57      0.61      0.59        28
           1       0.60      0.75      0.67        28
           2       0.67      0.07      0.12        29
           3       0.00      0.00      0.00        25
           4       0.67      0.18      0.29        22
           5       0.50      0.05      0.08        22
           6       0.58      0.42      0.49        26
           7       0.47      0.30      0.36        27
           8       1.00      0.04      0.07        27
           9       0.50      0.04      0.07        28
          10       0.40      0.72      0.51       179

    accuracy                           0.44       441
   macro avg       0.54      0.29      0.30       441
weighted avg       0.49      0.44      0.37       441

Logger {'time': {0: 0.0497589111328125}, 'loss': {0: 1.401717185974121}, 'F1': {30: 0.2952274392953682},

# Train without batches

In [72]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, f1_score


def train_model(model, train_data, criterion, optimizer, num_epochs, test_data):
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        logger = {'time': {}, 'loss': {}, 'F1': {}, 'Accuracy': {}}

        # Iterate over each data point in the training data
        for i, (sequence, label) in enumerate(train_data):
            # Convert label to tensor if it's not already
            if not isinstance(label, torch.Tensor):
                label = torch.tensor(label, dtype=torch.long)

            # Forward pass
            outputs = model(sequence.unsqueeze(0))
            loss = criterion(outputs, label.unsqueeze(0))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            if i % 100 == 0:
                logger['time'][i] = time.time() - epoch_start_time
                logger['loss'][i] = loss.item()

        epoch_end_time = time.time()
        print(f"Epoch {epoch + 1}\n")
        print(f"Epoch time: {epoch_end_time - epoch_start_time}")

        # Evaluate at the end of the epoch
        f1_score_value, accuracy = evaluate_model(model, test_data)
        logger['F1'][epoch + 1] = f1_score_value
        logger['Accuracy'][epoch + 1] = accuracy

        # Print detailed logger information
        print("Logger", logger)

def evaluate_model(model, test_data):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for sequence, label in test_data:
            # Ensure label is a tensor
            if not isinstance(label, torch.Tensor):
                label = torch.tensor(label, dtype=torch.long)

            outputs = model(sequence.unsqueeze(0))
            _, predicted = torch.max(outputs.data, 1)

            total += 1  # Increment total for each label
            correct += (predicted == label).sum().item()
            all_predictions.append(predicted.item())
            all_labels.append(label.item())

    print(classification_report(all_labels, all_predictions))
    f1_score_value = f1_score(all_labels, all_predictions, average='macro')
    accuracy = correct / total
    return f1_score_value, accuracy

# Load and shuffle MNIST dataset
train_data, test_data = load_and_shuffle_mnist()

# Set LSTM parameters
input_size = 784  # 28x28
hidden_size = 128
num_classes = 11  # Digits 0-9 and 'null'
num_epochs = 10
batch_size = 100
learning_rate = 0.001
window_size = 10  # Example window size

# Create model
model = SimpleLSTM(input_size, hidden_size, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Example of training with a single data point
for size in [1000, 5000, 10000, 20000, 30000, 40000, 50000]:
    print(f"Training with dataset size: {size}")
    sampled_train_data = balanced_sampling(train_data, size)
    sampled_test_data = balanced_sampling(test_data, size // 2)

    train_sequences, train_labels = create_sequences(sampled_train_data, window_size)
    test_sequences, test_labels = create_sequences(sampled_test_data, window_size)

    # Convert sequences and labels into a list of tuples for easier iteration
    train_data_tuples = list(zip(train_sequences, train_labels))
    test_data_tuples = list(zip(test_sequences, test_labels))

    train_model(model, train_data_tuples, criterion, optimizer, num_epochs, test_data_tuples)


Training with dataset size: 1000
balanced_sampling
Done sampling.
balanced_sampling
Done sampling.
Epoch 1

Epoch time: 4.595603942871094


/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.59      0.59      0.59        27
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        27
           4       0.00      0.00      0.00        28
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00        25
           7       0.00      0.00      0.00        28
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        28
          10       0.37      0.93      0.53       164

    accuracy                           0.38       441
   macro avg       0.09      0.14      0.10       441
weighted avg       0.17      0.38      0.23       441

Logger {'time': {0: 0.015468835830688477, 100: 0.5594398975372314, 200: 1.0551037788391113, 300: 1.6020641326904297, 400: 2.1452648639678955, 500: 2.70550274848938, 600: 3.195931673049926

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.58      0.56      0.57        27
           1       0.00      0.00      0.00        28
           2       0.60      0.10      0.18        29
           3       0.50      0.15      0.23        27
           4       0.59      0.36      0.44        28
           5       0.00      0.00      0.00        30
           6       0.65      0.60      0.63        25
           7       0.00      0.00      0.00        28
           8       0.37      0.26      0.30        27
           9       0.00      0.00      0.00        28
          10       0.36      0.75      0.49       164

    accuracy                           0.40       441
   macro avg       0.33      0.25      0.26       441
weighted avg       0.34      0.40      0.32       441

Logger {'time': {0: 0.006639242172241211, 100: 0.4898984432220459, 200: 0.9854342937469482, 300: 1.4615452289581299, 400: 2.018139362335205, 500: 2.5223193168640137, 600: 3.05710721015930

...

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.58      0.70      0.63        27
           1       0.00      0.00      0.00        28
           2       0.71      0.69      0.70        29
           3       0.60      0.44      0.51        27
           4       0.58      0.25      0.35        28
           5       0.62      0.17      0.26        30
           6       0.60      0.60      0.60        25
           7       0.64      0.32      0.43        28
           8       0.52      0.48      0.50        27
           9       0.56      0.36      0.43        28
          10       0.37      0.58      0.45       164

    accuracy                           0.46       441
   macro avg       0.53      0.42      0.44       441
weighted avg       0.48      0.46      0.44       441

Logger {'time': {0: 0.0058460235595703125, 100: 0.5240237712860107, 200: 1.0793049335479736, 300: 1.631138801574707, 400: 2.1448137760162354, 500: 2.6149168014526367, 600: 3.0985689163208

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.58      0.78      0.67        27
           1       0.00      0.00      0.00        28
           2       0.73      0.66      0.69        29
           3       0.65      0.56      0.60        27
           4       0.69      0.32      0.44        28
           5       0.60      0.10      0.17        30
           6       0.56      0.40      0.47        25
           7       0.62      0.46      0.53        28
           8       0.56      0.52      0.54        27
           9       0.60      0.32      0.42        28
          10       0.38      0.60      0.46       164

    accuracy                           0.48       441
   macro avg       0.54      0.43      0.45       441
weighted avg       0.49      0.48      0.45       441

Logger {'time': {0: 0.00569605827331543, 100: 0.5752592086791992, 200: 1.130937099456787, 300: 1.626847267150879, 400: 2.136575222015381, 500: 2.6297051906585693, 600: 3.130347967147827, 

KeyboardInterrupt: 

In [76]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, f1_score


def train_model(model, train_data, criterion, optimizer, num_epochs, test_data, eval_interval=100):
    logger = {'time': {}, 'loss': {}, 'F1': {}, 'Accuracy': {}}

    for epoch in range(num_epochs):
        epoch_start_time = time.time()

        for i, (sequence, label) in enumerate(train_data):
            # Convert sequence and label to tensors if they are not already
            if not isinstance(sequence, torch.Tensor):
                sequence = torch.tensor(sequence, dtype=torch.float32)
            if not isinstance(label, torch.Tensor):
                label = torch.tensor(label, dtype=torch.long)

            # Forward pass
            outputs = model(sequence.unsqueeze(0))
            loss = criterion(outputs, label.unsqueeze(0))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            if i % 100 == 0:
                logger['time'][i] = time.time() - epoch_start_time
                logger['loss'][i] = loss.item()

                # Evaluate model performance on test data
                f1_score_value, accuracy = evaluate_model(model, test_data)
                logger['F1'][i] = f1_score_value
                logger['Accuracy'][i] = accuracy

                print(f"Epoch {epoch + 1}, Step {i}, Loss: {loss.item()}, F1: {f1_score_value}, Accuracy: {accuracy}, Time Elapsed: {time.time() - epoch_start_time} seconds")

        epoch_end_time = time.time()
        print(f"Epoch {epoch + 1} completed. Time: {epoch_end_time - epoch_start_time}")

        # Print detailed logger information
        print("Logger", logger)
        
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import numpy as np

def evaluate_model(model, test_data):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for sequence, label in test_data:
            # Ensure label is a tensor
            if not isinstance(label, torch.Tensor):
                label = torch.tensor(label, dtype=torch.long)

            outputs = model(sequence.unsqueeze(0))
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.append(predicted.item())
            all_labels.append(label.item())

    # Calculate confusion matrix and metrics
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
    f1_score_value = f1_score(all_labels, all_predictions, average='macro')

    # Print classification report
    print(classification_report(all_labels, all_predictions))

    return f1_score_value, accuracy

# Load and shuffle MNIST dataset
train_data, test_data = load_and_shuffle_mnist()

# Set LSTM parameters
input_size = 784  # 28x28
hidden_size = 128
num_classes = 11  # Digits 0-9 and 'null'
num_epochs = 5
batch_size = 100
learning_rate = 0.001
window_size = 10  # Example window size

# Create model
model = SimpleLSTM(input_size, hidden_size, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Example of training with a single data point
for size in [32000]:
    print(f"Training with dataset size: {size}")
    sampled_train_data = balanced_sampling(train_data, size)
    sampled_test_data = balanced_sampling(test_data, size // 2)

    train_sequences, train_labels = create_sequences(sampled_train_data, window_size)
    test_sequences, test_labels = create_sequences(sampled_test_data, window_size)

    # Convert sequences and labels into a list of tuples for easier iteration
    train_data_tuples = list(zip(train_sequences, train_labels))
    test_data_tuples = list(zip(test_sequences, test_labels))

    train_model(model, train_data_tuples, criterion, optimizer, num_epochs, test_data_tuples)


Training with dataset size: 32000
balanced_sampling
Done sampling.
balanced_sampling
Done sampling.


/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.06      1.00      0.12       912
           1       0.00      0.00      0.00       885
           2       0.00      0.00      0.00       877
           3       0.00      0.00      0.00       897
           4       0.00      0.00      0.00       892
           5       0.00      0.00      0.00       862
           6       0.00      0.00      0.00       903
           7       0.00      0.00      0.00       889
           8       0.00      0.00      0.00       892
           9       0.00      0.00      0.00       876
          10       0.00      0.00      0.00      5646

    accuracy                           0.06     14531
   macro avg       0.01      0.09      0.01     14531
weighted avg       0.00      0.06      0.01     14531

Epoch 1, Step 0, Loss: 2.4203009605407715, F1: 0.01073743325896405, Accuracy: 0.06276237010529213, Time Elapsed: 14.67314100265503 seconds


/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       912
           1       0.00      0.00      0.00       885
           2       0.00      0.00      0.00       877
           3       0.00      0.00      0.00       897
           4       0.00      0.00      0.00       892
           5       0.00      0.00      0.00       862
           6       0.00      0.00      0.00       903
           7       0.00      0.00      0.00       889
           8       0.00      0.00      0.00       892
           9       0.00      0.00      0.00       876
          10       0.39      1.00      0.56      5646

    accuracy                           0.39     14531
   macro avg       0.04      0.09      0.05     14531
weighted avg       0.15      0.39      0.22     14531

Epoch 1, Step 100, Loss: 3.5031092166900635, F1: 0.05087701117834438, Accuracy: 0.38854862019131514, Time Elapsed: 30.499523162841797 seconds


...

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       912
           1       0.57      0.97      0.72       885
           2       0.58      0.73      0.64       877
           3       0.60      0.66      0.63       897
           4       0.60      0.45      0.52       892
           5       0.55      0.40      0.47       862
           6       0.63      0.62      0.63       903
           7       0.57      0.73      0.64       889
           8       0.61      0.42      0.49       892
           9       0.58      0.71      0.64       876
          10       0.39      0.41      0.40      5646

    accuracy                           0.51     14531
   macro avg       0.52      0.55      0.52     14531
weighted avg       0.47      0.51      0.48     14531

Epoch 5, Step 26000, Loss: 0.4485699534416199, F1: 0.5243253405153473, Accuracy: 0.505058151538091, Time Elapsed: 21130.747053146362 seconds
              precision    recall  f1-score   

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       912
           1       0.60      0.91      0.72       885
           2       0.44      0.89      0.59       877
           3       0.58      0.75      0.66       897
           4       0.60      0.42      0.50       892
           5       0.54      0.40      0.46       862
           6       0.60      0.82      0.69       903
           7       0.57      0.85      0.68       889
           8       0.57      0.62      0.60       892
           9       0.60      0.56      0.58       876
          10       0.39      0.32      0.35      5646

    accuracy                           0.50     14531
   macro avg       0.50      0.59      0.53     14531
weighted avg       0.46      0.50      0.47     14531

Epoch 5, Step 26200, Loss: 1.2926077842712402, F1: 0.5298624927689641, Accuracy: 0.5041635124905375, Time Elapsed: 21170.266030073166 seconds
              precision    recall  f1-score  

/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/penny_varsou/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.61      0.37      0.46       912
           1       0.61      0.85      0.71       885
           2       0.57      0.66      0.61       877
           3       0.00      0.00      0.00       897
           4       0.57      0.79      0.67       892
           5       0.51      0.18      0.26       862
           6       0.60      0.76      0.67       903
           7       0.60      0.82      0.69       889
           8       0.57      0.60      0.58       892
           9       0.58      0.59      0.59       876
          10       0.39      0.41      0.40      5646

    accuracy                           0.50     14531
   macro avg       0.51      0.55      0.51     14531
weighted avg       0.47      0.50      0.48     14531

Epoch 5, Step 27900, Loss: 0.4729008376598358, F1: 0.5131892583172074, Accuracy: 0.5039570573257174, Time Elapsed: 21428.494715213776 seconds
              precision    recall  f1-score  